# 1：下载没有的库

In [3]:
#!mkdir /home/aistudio/external-libraries
#!pip install visualdl -t /home/aistudio/external-libraries
#!pip install d2l -t /home/aistudio/external-libraries

# 2：导入所需要的库

In [1]:
import sys
sys.path.append('/home/aistudio/external-libraries')
%matplotlib inline
import os
import paddle
import paddle.vision
from paddle.nn import functional as F
from paddle import nn
from d2l import paddle as d2l
from paddle.vision.datasets.folder import default_loader

### 2.1：加入可视化模块，记得改名字！

In [2]:
# 导入VisualDL的包
from visualdl import LogWriter
logw = LogWriter("./random_log/log—1", sync_cycle=10)

# 3：文件解压

In [6]:
import zipfile
def un_zip(file_name, dst):
    """解压 zip 文件"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(dst):
        pass
    else:
        os.mkdir(dst)
    for names in zip_file.namelist():
        zip_file.extract(names, dst)
    zip_file.close()



file_name = r"/home/aistudio/data/data183509/release_data.zip"
dst = r"./work/"
#un_zip(file_name, dst)


# 4：定义数据集

In [3]:
class readImageDataset(paddle.io.Dataset):
    def __init__(self, state, target_transform=None):
        super().__init__()
        self.state = state
        self.root = r"./data/release_data"  # 该文件夹内包含train、val以及test三个文件夹
        self.labels, self.paths = self.getDataPath()
        self.target_transform = target_transform

    # 定义一个读取数据集的子类
    def getDataPath(self):
        paths = []
        labels = []
        assert self.state == 'train' or self.state == 'val' or self.state == 'test'  # 用于选取所读指定的数据集
        if self.state == 'train':
            f_path = r'./work/release_data/train_list.txt'#train模式文件地址
            path_compose=r'train'

        if self.state == 'val':
            f_path = r'./work/release_data/val_list.txt'
            path_compose = r'val'

        if self.state == 'test':
            f_path = r'./work/release_data/test_truth_list.txt'
            path_compose = r'test'

        with open(f_path) as f:
            line = f.readline()
            while line:
                if "006626.jpg" not in line:
                   linestr = line.split(' ')  # 一行有多个数据，以空格为间隔将字符串隔开读取
                   paths.append(os.path.join(self.root,path_compose, linestr[0]))
                         # 读取该行的第一个数据
                   labels.append(int(linestr[1]))
                line = f.readline()  # 从下一行读取一个新的值出来
        return (labels, paths)


    def __getitem__(self, index):
        path = self.paths[index]
        label = self.labels[index]
        image = default_loader(path)
        if self.target_transform is not None:
            try:
                image = self.target_transform(image)
            except IOError:
                print("多余的地址：",path)
                self.__getitem__(10)
        return image,label

    def __len__(self):
        return len(self.labels)

# 5：图像预处理

In [4]:
normalize = paddle.vision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = paddle.vision.transforms.Compose([
    paddle.vision.transforms.RandomResizedCrop(380),
    paddle.vision.transforms.RandomHorizontalFlip(),
    paddle.vision.transforms.ToTensor(),
    normalize])

test_augs = paddle.vision.transforms.Compose([
    paddle.vision.transforms.Resize(420),
    paddle.vision.transforms.CenterCrop(380),
    paddle.vision.transforms.ToTensor(),
    normalize])

To_Tensor=paddle.vision.transforms.ToTensor()
transform=paddle.vision.transforms.Compose([To_Tensor,paddle.vision.transforms.Resize(96)])
train_datas=readImageDataset('train',target_transform=train_augs)
test_datas=readImageDataset('test',target_transform=test_augs)

# 5：加载预训练网络模型

In [10]:
finetune_net = paddle.vision.models.wide_resnet50_2(pretrained=True)
finetune_net.fc = nn.Sequential(nn.Linear(finetune_net.fc.state_dict()['weight'].shape[0], 300),nn.ReLU(),nn.Dropout(0.4),nn.Linear(300,208))
for layer in finetune_net.fc:
    if type(layer) == nn.Linear:
        nn.initializer.XavierUniform(layer.state_dict()['weight'])
print(finetune_net)

ResNet(
  (conv1): Conv2D(3, 64, kernel_size=[7, 7], stride=[2, 2], padding=3, data_format=NCHW)
  (bn1): BatchNorm2D(num_features=64, momentum=0.9, epsilon=1e-05)
  (relu): ReLU()
  (maxpool): MaxPool2D(kernel_size=3, stride=2, padding=1)
  (layer1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv2D(64, 128, kernel_size=[1, 1], data_format=NCHW)
      (bn1): BatchNorm2D(num_features=128, momentum=0.9, epsilon=1e-05)
      (conv2): Conv2D(128, 128, kernel_size=[3, 3], padding=1, data_format=NCHW)
      (bn2): BatchNorm2D(num_features=128, momentum=0.9, epsilon=1e-05)
      (conv3): Conv2D(128, 256, kernel_size=[1, 1], data_format=NCHW)
      (bn3): BatchNorm2D(num_features=256, momentum=0.9, epsilon=1e-05)
      (relu): ReLU()
      (downsample): Sequential(
        (0): Conv2D(64, 256, kernel_size=[1, 1], data_format=NCHW)
        (1): BatchNorm2D(num_features=256, momentum=0.9, epsilon=1e-05)
      )
    )
    (1): BottleneckBlock(
      (conv1): Conv2D(256, 128, kernel_siz

## 5.0：新版加载预训练模型

In [7]:
model=paddle.jit.load("./work/inference")

In [11]:
    print(model.program)

<bound method TranslatedLayer.program of TranslatedLayer()>


## 5.1是否用自己的参数，记得更改加载的路径

In [ ]:
#finetune_net.set_state_dict(paddle.load('net_dict_6.pdparams'))

# 6：训练前定义训练的函数

In [ ]:
def train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus(),records=None):
    """Defined in :numref:`sec_image_augmentation`"""
    """用多GPU进行模型训练
    Defined in :numref:`sec_image_augmentation`"""
    step=0
    timer, num_batches = d2l.Timer(), len(train_iter)
    net = paddle.DataParallel(net)
    for epoch in range(num_epochs):
        # 4个维度：储存训练损失，训练准确度，实例数，特点数
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            l, acc = train_batch_ch13(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            step=step+1
            if (i + 1) % (num_batches // 10) == 0 or i == num_batches - 1:
                logw.add_scalar('loss',metric[0] / metric[2], step)
                logw.add_scalar('train_acc',metric[1] / metric[3], step)
                print(epoch + (i + 1) / num_batches)   
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        logw.add_scalar('test_acc', test_acc, step)
        print('epoc:', epoch + (i + 1) / num_batches, 'loss:', metric[0] / metric[2], 'train_acc:',
              metric[1] / metric[3],'test_acc', test_acc)
        print("\ntime:",datetime.datetime.now())
    print(f'loss {metric[0] / metric[2]:.3f}, train acc '
          f'{metric[1] / metric[3]:.3f}, test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec on '
          f'{str(devices)}')

In [ ]:
def train_fine_tuning(net, learning_rate, batch_size=128, num_epochs=25,
                      param_group=True):
    train_iter = paddle.io.DataLoader(train_datas,
        batch_size=batch_size, shuffle=True)
    test_iter = paddle.io.DataLoader(test_datas,
        batch_size=batch_size)
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.0.weight", "fc.0.bias","fc.3.weight", "fc.3.bias"]]
        trainer = paddle.optimizer.SGD(learning_rate=learning_rate, parameters=[{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'learning_rate': learning_rate*10}],
                                    weight_decay=0.001)
    else:
        trainer = paddle.optimizer.SGD(learning_rate=learning_rate, parameters=net.parameters(), 
                                  weight_decay=0.001)
    train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,devices,records=logw)

# 7：开始训练

In [ ]:
train_fine_tuning(finetune_net, 1e-3)

# 8：保存模型，记得改名字

In [ ]:
paddle.save(finetune_net.state_dict(), 'net_dict_one.pdparams')

# 9：预测函数

## 9.1 把标签变成人话

In [6]:
import openpyxl
def xlsx_trans(root):
    mybook=openpyxl.load_workbook(root,data_only=True)
    mySheet=mybook.active
    #按行获取新书表的单元格（第一行除外--标题，不是数据）
    myRows=list(mySheet.values)[:]
    mydics={}
    for myRow in myRows:
        mydics[myRow[0]]=myRow[1]
    return mydics
mydicts=xlsx_trans("/home/aistudio/data/data183509/class_names.xlsx")

## 9.2 定义预测函数

In [70]:
def predict(path,transform,net,label_read,top_x):
    list=[]
    list1=[]
    img=default_loader(path)
    img_tensor=transform(img)
    img_tensor=paddle.unsqueeze(img_tensor,axis=0)
    net.eval()
    x=net(img_tensor)
    xl=(x.numpy().tolist())[0]
    temp={}
    for i in range(top_x):
      temp[mydicts[int(xl.index(max(xl)))]] = max(xl)
      # list.append(xl.index(max(xl)))
      xl[xl.index(max(xl))]=0
    # for i in list:
    #     list1.append(mydicts[int(i)])
    # if top_x == 1:
    return temp
    # else:
    #   return list1


def predict_list(path_list,transform,net,label_read,top_x):
  list=os.listdir(path_list)
  result=[]
  result1=[]
  for i in list:
    if "jpg" in i:
      path=os.path.join(path_list,i)
      result.append(predict(path,transform,net,label_read,top_x))
      result1.append(i)
  return (result,result1)



## 文件夹整体预测

In [80]:
path=r"/home/aistudio/work/release_data/hi"
top_3s=predict_list(path,test_augs,model,mydicts,top_x=3)
for i in range(len(top_3s[1])):
    print(top_3s[0][i],top_3s[1][i],"\n")

{'凉拌凉面': 0.5653793811798096, '茄汁拌面': 0.08540446311235428, '炒面': 0.07997732609510422} 炒面.jpg 

{'莲藕排骨汤': 0.4674493074417114, '莲藕': 0.3581307828426361, '冬瓜汤': 0.009066015481948853} 莲藕排骨汤.jpg 

{'冰糖雪梨': 0.9577760696411133, '银耳汤': 0.0034847038332372904, '清炒山药': 0.001095216372050345} 冰糖雪梨.jpg 

{'鱼香肉丝': 0.930090606212616, '木耳炒肉丝': 0.010059439577162266, '莴笋肉丝': 0.0028328446205705404} 鱼香肉丝.jpg 

{'大米粥': 0.5422576069831848, '西湖牛肉羹': 0.15969528257846832, '皮蛋瘦肉粥': 0.09777914732694626} 西湖牛肉羹.jpg 

{'鱼香茄子': 0.8916314840316772, '肉末茄子': 0.038121841847896576, '蒜泥茄子': 0.005707074888050556} 鱼香茄子.jpg 

{'大米粥': 0.39374715089797974, '红薯粥': 0.17208372056484222, '小米粥': 0.10852274298667908} 大米粥.jpg 

{'红薯粥': 0.6780503988265991, '大米粥': 0.2483544945716858, '小米粥': 0.019263584166765213} 红薯粥.jpg 

{'水煮鱼': 0.28039926290512085, '毛血旺': 0.19879266619682312, '皮皮虾': 0.07771693915128708} 皮皮虾.jpg 



## 单个文件预测

In [81]:
path=r"/home/aistudio/work/release_data/hi/虾.jpg"
top_5s=predict(path, test_augs,model,mydicts, top_x=5)
print(top_5s)

{'油焖大虾': 0.834388256072998, '香辣虾': 0.03609723225235939, '包子': 0.0033816404175013304, '肉酱意大利面': 0.0026113640051335096, '皮皮虾': 0.002368600806221366}


# 10：自己玩